# Reconciliation Monthly Stats

We need to load in these libraries into our notebook in order to query, load, manipulate and view the data

In [ ]:
import os
from datetime import datetime, timedelta
from config import Config
import requests

%load_ext sql
%config SqlMagic.displaylimit = 5

# Parameters cell for external parameters via papermill (job running this notebook will insert a parameter cell below this). This cell has a tag of with the name "parameters" that is used by papermill

e.g.
param1 = "some_value"

This will create the connection to the database and prep the jupyter magic for SQL

In [ ]:
%sql $Config.SQLALCHEMY_DATABASE_URI

Simplest query to run to ensure our libraries are loaded and our DB connection is working

In [ ]:
%%sql
set time zone 'UTC';

Query ...

In [ ]:
%%sql monthly_reconciliation_summary  <<
SELECT
sum(pli.total)                    AS subtotal,
sum(pli.service_fees)             AS service_fees,
sum(pli.total + pli.service_fees) AS total,
(i.payment_date AT TIME ZONE 'UTC' AT TIME ZONE 'America/Vancouver')::date,
pli.description,
i.payment_method_code,
i.corp_type_code
FROM invoices i
JOIN payment_line_items pli ON i.id = pli.invoice_id
WHERE i.corp_type_code = :partner_code
AND i.invoice_status_code = 'PAID'
AND i.payment_method_code IN ('PAD', 'EJV', 'DIRECT_PAY', 'DRAWDOWN')
AND date(i.payment_date AT TIME ZONE 'UTC' AT TIME ZONE 'America/Vancouver') > date(current_date - 1 - interval '1 months')
AND date(i.payment_date AT TIME ZONE 'UTC' AT TIME ZONE 'America/Vancouver') <= date(current_date - 1)
GROUP BY (i.payment_date AT TIME ZONE 'UTC' AT TIME ZONE 'America/Vancouver')::date, i.payment_method_code, i.corp_type_code, pli.description
ORDER BY (i.payment_date AT TIME ZONE 'UTC' AT TIME ZONE 'America/Vancouver')::date, pli.description, i.payment_method_code

Save to CSV

In [ ]:
filename_summary = os.path.join(os.getcwd(), r'data/')+partner_code+'_monthly_reconciliation_summary_' + datetime.strftime(datetime.now()-timedelta(1), '%Y-%m') +'.csv'
df_summary = monthly_reconciliation_summary.DataFrame()
with open(filename_summary, 'w') as f:
    f.write('Monthly Reconciliation Summary:\n\n')
    if df_summary.empty:
        f.write('No Data Retrieved')
    else:
        df_summary.to_csv(f, sep=',', encoding='utf-8', index=False)

In [ ]:
%%sql monthly_reconciliation_disbursed  <<
SELECT count(*)                   AS transaction_count,
sum(pli.total)                    AS sub_total,
sum(pli.service_fees)             AS service_fees,
sum(pli.total + pli.service_fees) AS total,
(i.disbursement_date AT TIME ZONE 'UTC' AT TIME ZONE 'America/Vancouver')::date,
pli.description,
i.payment_method_code,
i.corp_type_code
FROM invoices i
JOIN payment_line_items pli ON i.id = pli.invoice_id
WHERE i.corp_type_code = 'VS'
AND i.invoice_status_code = 'PAID'
AND i.payment_method_code IN ('PAD', 'EJV', 'DIRECT_PAY')
AND i.disbursement_status_code = 'COMPLETED'
AND date(disbursement_date AT TIME ZONE 'UTC' AT TIME ZONE 'America/Vancouver') > date(current_date - 1 - interval '1 months')
AND date(disbursement_date AT TIME ZONE 'UTC' AT TIME ZONE 'America/Vancouver') <= date(current_date - 1)
GROUP BY (disbursement_date AT TIME ZONE 'UTC' AT TIME ZONE 'America/Vancouver')::date, payment_method_code, corp_type_code, pli.description
ORDER BY (disbursement_date AT TIME ZONE 'UTC' AT TIME ZONE 'America/Vancouver')::date, pli.description, i.payment_method_code;

Save to another CSV

In [ ]:
filename_disbursed = os.path.join(os.getcwd(), r'data/')+partner_code+'_monthly_reconciliation_disbursed_' + datetime.strftime(datetime.now()-timedelta(1), '%Y-%m') +'.csv'
df_disbursed = monthly_reconciliation_disbursed.DataFrame()
with open(filename_disbursed, 'a') as f:
    f.write('Monthly Reconciliation Disbursed:\n\n')
    if df_disbursed.empty:
        f.write('No Data Retrieved')
    else:
        df_disbursed.to_csv(f, sep=',', encoding='utf-8', index=False)


In [ ]:
# Get the API base URL from the environment variable
API_BASE_URL = os.getenv('REPORT_API', '')
if not API_BASE_URL:
    raise ValueError("The REPORT_API environment variable is not set or is empty")

url = API_BASE_URL
headers = {
    'Authorization': 'Bearer your_jwt_token',
    'Content-Type': 'application/json',
    'Accept': 'application/pdf'
}

# SQL query
query = """
SELECT
    COUNT(*) AS transaction_count,
    SUM(total) AS total,
    TO_CHAR(DATE_TRUNC('month',current_date) - INTERVAL '1 month','Month') as month,
    corp_type_code
FROM 
    invoices
WHERE 
    corp_type_code = :partner_code
    AND invoice_status_code = 'PAID'
    AND payment_method_code IN ('PAD', 'EJV')
    AND DATE_TRUNC('month', created_on AT TIME ZONE 'UTC' AT TIME ZONE 'America/Vancouver') = DATE_TRUNC('month', current_date - INTERVAL '1 month')
GROUP BY 
    corp_type_code
ORDER BY 
    month;
"""

# Execute the SQL query and fetch results
result = %sql $query

# Print the result to verify
print(result)

# Check if the result is not None
if result:
    # Convert the result to a DataFrame
    df = result.DataFrame()

    # Rename columns to match the expected names in the template
    df.rename(columns={
        'corp_type_code': 'Registry',
        'transaction_count': 'Trans_Counts',
        'total': 'Amount'
    }, inplace=True)

    # Add a Symbol column with a dollar sign
    df['Symbol'] = '$'

    # Convert DataFrame to JSON-compatible format
    table_rows = df.to_dict(orient='records')

    # Get the current date
    current_date = datetime.now().strftime("%B %d, %Y")

    # Define the request body
    data = {
        "templateVars": {
            "Date": current_date,
            "Company_Name": "Vital Statistics Agency",
            "Address_Line_1": "PO Box 9657, Stn Prov Govt",
            "Address_Line_2": "",
            "City": "VICTORIA",
            "Province": "BC",
            "Area_Code": "V8W 9P3",
            "First_Name": partner_code,
            "enter_month": df['month'][0] if not df.empty else "N/A",
            "table_rows": table_rows
        },
        "templateName": "revenue_letter",
        "reportName": "revenue_letter"
    }

    # Send the POST request
    response = requests.post(url, headers=headers, json=data)

    # Check if the response is successful
    if response.status_code == 200:
        # Get the PDF content from the response
        pdf_content = response.content
        
        # Save the PDF content to a file
        with open(partner_code+'_revenue_letter.pdf', 'wb') as pdf_file:
            pdf_file.write(pdf_content)
        
        print("PDF report saved successfully as 'payment_receipt.pdf'")
    else:
        print('Failed to get the report:', response.text)
else:
    print('No results returned from the SQL query')